In [1]:
import subprocess
import os
import paramiko
import ipywidgets as widgets
from IPython.display import display

# テキストボックスの作成
array_input = widgets.Text(
    value='',
    placeholder='配列を入力してください',
    description='配列:',
)

# ボタンの作成
run_button = widgets.Button(
    description='実行',
    button_style='success'
)

# 出力エリアの作成
output_area = widgets.Output()

# テキストボックスとボタン、出力エリアの表示
display(array_input, run_button, output_area)

# 実行ボタンが押されたときに呼び出される関数
def on_button_click(b):
    with output_area:
        output_area.clear_output()  # 以前の出力をクリア
        value = array_input.value
        print(f"入力された配列: {value}")
        
        # 必要な変数をここで定義
        segid = "PROA"
        rename_str = "rename_str_example"
        delete_str = "delete_str_example"
        cycpep = "cycpep_example"

        # GGAAなど、完全な配列に対応するディレクトリのみを作成
        functions = [
            (copy_seed_work, [value]),
            (copy_seed_pdb, [value])
        ]
        
        for func, args in functions:
            try:
                func(*args)
            except NameError:
                print(f"Error: {func.__name__} 関数が定義されていません。")
            except Exception as e:
                print(f"An error occurred in {func.__name__}: {e}")
                
        try:
            seed_charmm(value)
        except NameError:
            print("Error: seed_charmm 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in seed_charmm: {e}")

        try:
            in_charminp(value)
        except NameError:
            print("Error: in_charminp 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in in_charminp: {e}")

        try:
            connect_server()
        except NameError:
            print("Error: connect_server 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in connect_server: {e}")

        try:
            send_dir(value)
        except NameError:
            print("Error: send_dir 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in send_dir: {e}")
# ボタンがクリックされたときに実行されるように設定
run_button.on_click(on_button_click)

#try:
 #           sftp, local_path, remote_path = send_dir(value)
  #          upload_directory(sftp, local_path, remote_path)
  #      except NameError:
  #          print("Error: upload_directory 関数が定義されていません。")
  #      except Exception as e:
   #         print(f"An error occurred in upload_directory: {e}")

Text(value='', description='配列:', placeholder='配列を入力してください')

Button(button_style='success', description='実行', style=ButtonStyle())

Output()

In [2]:
#配列を入力すると配列名に合わせてMD計算に使うディレクトリをコピーしてくれるコマンド
def copy_seed_work(value):
    # コピー元とコピー先のパスを定義
    source_path = "/Users/teraimao/experiment/work/seed_work"
    destination_path = f"/Users/teraimao/experiment/work/{value}_work"
    
    # ディレクトリが存在するか確認
    import os
    if not os.path.exists(source_path):
        print(f"Error: The source directory '{source_path}' does not exist.")
        return
    
    # コピーコマンドの実行
    #subprocess.run(["cp", "-r", source_path, destination_path], check=True)
    try:
        subprocess.run(["cp", "-r", source_path, destination_path], check=True)
        print(f"Directory copied successfully to '{destination_path}'.")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred during copying: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


In [3]:
#コピーしたディレクトリに配列数に合わせたpdbファイルをコピーするコマンド2
def copy_seed_pdb(value):
    #source_path = f"/Users/teraimao/experiment/data/seed_pdb/gl4.pdb"
    destination_path = f"/Users/teraimao/experiment/work/{value}_work/"
    #
    try:
        if len(value)== 4:
            source_path = "/Users/teraimao/experiment/data/seed_pdb/gl4.pdb"
            subprocess.run(["cp", source_path, destination_path], check=True)
            return
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while copying the file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [4]:
import os

def seed_charmm(value):
    segid = "PROA"
    kind = {'G':'GLY','A':'ALA','V':'VAL','L':'LEU','I':'ILE','C':'CYS',
            'M':'MET','S':'SEL','T':'THR','D':'ASP','E':'GLN','R':'ARG',
            'K':'LYS','H':'HIS','F':'PHE','Y':'TYR','W':'TRP','P':'PRO'}

    rename_commands = []
    delete_commands = []
    
    for i, acid_code in enumerate(value):
        acid = kind.get(acid_code)
        if acid:
            num = i + 1
            rename = f'rename resname {acid} select segid {segid} .AND. resid {num} end'
            delete = f'delete atom select segid {segid} .and. resid {num} .and. -'
            rename_commands.append(rename)
            delete_commands.append(delete)
    
    last = len(value)
    cycpep = f'patch CYCPEP PROA 1 PROA {last} setup sort warn'
    
    rename_str = "\n".join(rename_commands)
    delete_str = "\n".join(delete_commands)

    if len(value)== 4:
        gly = 'gl4'
        

    # CHARMMスクリプトの生成
    charmm_script = f"""
wrnlev 5
prnlev 5 node 0 

open read card name ../toppar/4.1top_all36_prot.rtf unit 20
read rtf card unit 20
close unit 20

open read card name ../toppar/2_1par_all36m_prot.prm unit 20
read parameter card unit 20
close unit 20

open read card name ../{gly}.pdb unit 21
read sequence pdb unit 21

generate {segid} setup first NONE last NONE

autogen ANGLes DIHEdrals 

rewind unit 21

read coordinate pdb unit 21
close unit 21

!ic fill preserve
ic parameter
ic build
hbuild sele type H* end

{rename_str}

bomb -1

{delete_str}
 .not. (type n .or. type ca .or. type c .or. type o .or. type ha .or. type hn) end 

open write card name temp.pdb unit 14
write coordinates pdb select all end unit 14

close unit 14

delete atom select all end

open read card name temp.pdb unit 14
read sequence pdb unit 14
generate {segid} setup first NONE last NONE 
{cycpep}

rewind unit 14

read coordinate pdb unit 14
close unit 14

ic fill
ic parameter
ic build
hbuild sele type H* end

cons fix sele segid {segid} .and. (type N .or. type CA .or. type C .or. type O) end
MINI ABNR NSTEP 10000 NPRINT 100
cons fix clear

open unit 40 write form name "peptide.pdb"
write coor pdb unit 40
close unit 40

open unit 50 write form name "peptide.psf"
write psf card unit 50
close unit 50

system "rm temp.pdb"

stop
    """
    
    return charmm_script


In [5]:
import os

def seed_charmm(value):
    segid = "PROA"
    kind = {'G':'GLY','A':'ALA','V':'VAL','L':'LEU','I':'ILE','C':'CYS',
            'M':'MET','S':'SEL','T':'THR','D':'ASP','E':'GLN','R':'ARG',
            'K':'LYS','H':'HIS','F':'PHE','Y':'TYR','W':'TRP','P':'PRO'}

    rename_commands = []
    delete_commands = []
    
    for i, acid_code in enumerate(value):
        acid = kind.get(acid_code)
        if acid:
            num = i + 1
            rename = f'rename resname {acid} select segid {segid} .AND. resid {num} end'
            delete = f'delete atom select segid {segid} .and. resid {num} .and. -'
            rename_commands.append(rename)
            delete_commands.append(delete)
    
    last = len(value)
    cycpep = f'patch CYCPEP PROA 1 PROA {last} setup sort warn'
    
    rename_str = "\n".join(rename_commands)
    delete_str = "\n".join(delete_commands)

    if len(value)== 4:
        gly = 'gl4'
        

    # CHARMMスクリプトの生成
    charmm_script = f"""
wrnlev 5
prnlev 5 node 0 

open read card name ../toppar/4.1top_all36_prot.rtf unit 20
read rtf card unit 20
close unit 20

open read card name ../toppar/2_1par_all36m_prot.prm unit 20
read parameter card unit 20
close unit 20

open read card name ../{gly}.pdb unit 21
read sequence pdb unit 21

generate {segid} setup first NONE last NONE

autogen ANGLes DIHEdrals 

rewind unit 21

read coordinate pdb unit 21
close unit 21

!ic fill preserve
ic parameter
ic build
hbuild sele type H* end

{rename_str}

bomb -1

{delete_str}
 .not. (type n .or. type ca .or. type c .or. type o .or. type ha .or. type hn) end 

open write card name temp.pdb unit 14
write coordinates pdb select all end unit 14

close unit 14

delete atom select all end

open read card name temp.pdb unit 14
read sequence pdb unit 14
generate {segid} setup first NONE last NONE 
{cycpep}

rewind unit 14

read coordinate pdb unit 14
close unit 14

ic fill
ic parameter
ic build
hbuild sele type H* end

cons fix sele segid {segid} .and. (type N .or. type CA .or. type C .or. type O) end
MINI ABNR NSTEP 10000 NPRINT 100
cons fix clear

open unit 40 write form name "peptide.pdb"
write coor pdb unit 40
close unit 40

open unit 50 write form name "peptide.psf"
write psf card unit 50
close unit 50

system "rm temp.pdb"

stop
    """
    
    return charmm_script


In [6]:
#作成したcharmm.inpをコピーディレクトリに入れる関数
import os

def in_charminp(value):
    # CHARMMスクリプトを生成
    charmm = seed_charmm(value)
    # ディレクトリとファイルパスを設定
    directory = f"/Users/teraimao/experiment/work/{value}_work/charmm"
    destination_path = f"{directory}/charmm.inp"
    
    # ディレクトリが存在しない場合は作成
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # ファイルに書き込む
    with open(destination_path, 'w') as file:
        file.write(charmm)

In [7]:
#サーバーに接続する関数
def connect_server():
    hostname = '192.168.11.20'  # サーバーのホスト名
    username = 'teraimao'  # サーバーに接続するユーザー名
    password = '10Ap1128'  # ユーザーのパスワード
    client = paramiko.SSHClient()  # SSHクライアントのインスタンスを作成
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())  # ホストキーを自動的に追加
    client.connect(hostname, username=username, password=password)  # サーバーに接続
    return client  # 接続したクライアントを返す


In [8]:
#コピーディレクトリをサーバーに送る関数
def send_dir(value):
    client = connect_server()  # SSH接続を確立し、クライアントを取得
    sftp = client.open_sftp()  # SFTPセッションを開く
    local_path = f"/Users/teraimao/experiment/work/{value}_work/"  # ローカルのディレクトリ
    remote_path = f'/home/teraimao/experiment/work/{value}_work'  # リモートのディレクトリ

    # リモート側にディレクトリが存在しない場合は作成
    try:
        sftp.stat(remote_path)  # ディレクトリが存在するか確認
    except FileNotFoundError:
        sftp.mkdir(remote_path)  # 存在しない場合は作成

    # ディレクトリとファイルを再帰的にアップロード
    try:
        # ディレクトリとファイルを再帰的にアップロード
        upload_directory(sftp, local_path, remote_path)
    except Exception as e:
        print(f"An error occurred in upload_directory: {e}")

    try:
        run_csh(value)
    except Exception as e:
        print(f"An error occurred in run_csh: {e}")
    finally:
        sftp.close()  # SFTPセッションを閉じる
        client.close()  # SSHクライアントを閉じる


    # SFTPセッションとSSH接続をクローズ
    sftp.close()
    client.close()
    return sftp, local_path, remote_path

In [9]:
#コピーディレクトリの中身をサーバーに送る関数
def upload_directory(sftp, local_path, remote_path):
    for item in os.listdir(local_path):  # ローカルディレクトリ内のアイテムをリストアップ
        local_item = os.path.join(local_path, item)  # ローカルの完全パス
        remote_item = os.path.join(remote_path, item)  # リモートの完全パス

        if os.path.isdir(local_item):  # アイテムがディレクトリの場合
            try:
                sftp.stat(remote_item)  # リモートにディレクトリが存在するか確認
            except FileNotFoundError:
                sftp.mkdir(remote_item)  # 存在しない場合は作成
            upload_directory(sftp, local_item, remote_item)  # 再帰的にサブディレクトリをアップロード
        else:
            sftp.put(local_item, remote_item)  # ファイルをリモートにアップロード


In [10]:
#run.cshを実行するコマンド
def run_csh(value):
    client = connect_server() 
    remote_directory = f'/home/teraimao/experiment/work/{value}_work/charmm'

    try:
        # chmod +x Run.csh を実行して実行権限を付与
        chmod_command = f'cd {remote_directory} && chmod +x Run.csh'
        client.exec_command(chmod_command)

        # ./Run.csh を実行
        command = f'cd {remote_directory} && ./Run.csh'
        stdin, stdout, stderr = client.exec_command(command)
        output = stdout.read().decode()
        error = stderr.read().decode()

        # 標準出力とエラーを表示（必要に応じて処理）
        if output:
            print("Output:\n", output)
        if error:
            print("Error:\n", error)

    except Exception as e:
        print(f"An error occurred while executing the script: {e}")


In [1]:
import ipywidgets as widgets
from IPython.display import display

# テキスト入力ウィジェットの作成
array_input = widgets.Text(
    value='',
    placeholder='配列を入力してください',
    description='配列:',
)

# ボタンの作成
run_button = widgets.Button(
    description='実行',
    button_style='success'
)

# 出力エリアの作成
output_area = widgets.Output()

# テキストボックスとボタン、出力エリアの表示
display(array_input, run_button, output_area)

# 入力内容をリストに変換する関数
def get_list_from_input(value):
    global input
    with output_area:
        output_area.clear_output()
        input_text = array_input.value # テキストボックスから現在の入力を取得
        input = input_text.split(',')  # カンマで分割してリスト化
        print(f'リスト化：{input}')

# テキストウィジェットに変更があったときに呼び出す
run_button.on_click(get_list_from_input)


Text(value='', description='配列:', placeholder='配列を入力してください')

Button(button_style='success', description='実行', style=ButtonStyle())

Output()

In [12]:
input

<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x103d201f0>>

In [36]:
len(value)

5

In [2]:
#gbsw_in,sur,outの結果をpcに戻すコード
def return_log(value):
    client = connect_server()
    if client is None:
        print("Failed to connect to the server.")
        return
    
    peptide_position = ["in", "sur", "out"]
    try:
        for i in peptide_position:
            local_path = f'/home/teraimao/experiment/work/{value}_work/gbsw_{i}/log'
            remote_path = f'/Users/teraimao/experiment/work/{value}_work/gbsw_{i}'
            
            try:
                sftp.stat(remote_path)
            except FileNotFoundError:
                sftp.mkdir(remote_path)
            
            sftp.get(remote_file_path, local_file_path)
            print(f"Downloaded: {remote_file_path} -> {local_file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
def upload_directory(sftp, local_path, remote_path):
    for item in os.listdir(local_path):
        local_item = os.path.join(local_path, item)
        remote_item = os.path.join(remote_path, item)

        if os.path.isdir(local_item):
            try:
                sftp.stat(remote_item)
            except FileNotFoundError:
                sftp.mkdir(remote_item)
            upload_directory(sftp, local_item, remote_item)
        else:
            sftp.put(local_item, remote_item)


In [ ]:
#コピーディレクトリの中身をサーバーに送る関数
def upload_directory(sftp, local_path, remote_path):
    for item in os.listdir(local_path):
        local_item = os.path.join(local_path, item)
        remote_item = os.path.join(remote_path, item)

        if os.path.isdir(local_item):
            try:
                sftp.stat(remote_item)
            except FileNotFoundError:
                sftp.mkdir(remote_item)
            upload_directory(sftp, local_item, remote_item)
        else:
            sftp.put(local_item, remote_item)


In [ ]:
#計算済みコピーディレクトリをmacに戻すコマンド
def return_copy_work():
    client = connect_server()
    if client is None:
        print("Failed to connect to the server.")
        return
    
    sftp = client.open_sftp()
    local_path = f"/Users/teraimao/experiment/work/{value}_work/"
    remote_path =f"/Users/teraimao/experiment/work/{value}_work"
    
    with SCPClient(client.get_transport()) as scp:
            print(f"Downloading {remote_path} to {local_path} ...")
            scp.get(remote_path, local_path, recursive=True)  # recursive=Trueでディレクトリ対応
            print("Download complete!")

    except Exception as e:
        print(f"Error: {e}")

    finally:
        sftp.close()
        client.close()

In [ ]:
# 計算済みディレクトリをMacにコピーする関数
import paramiko
from scp import SCPClient

def return_copy_work(value):
    client = connect_server()
    if client is None:
        print("Failed to connect to the server.")
        return

    # リモートとローカルのパスを定義
    remote_path = f"/home/teraimao/experiment/work/{value}_work"
    local_path = f"/Users/teraimao/experiment/work/{value}_work/"

    try:
        with SCPClient(client.get_transport()) as scp:
            print(f"Downloading {remote_path} to {local_path} ...")
            scp.get(remote_path, local_path, recursive=True)  # recursive=Trueでディレクトリ対応
            print("Download complete!")

    except Exception as e:
        print(f"Error: {e}")

    finally:
        client.close()

In [ ]:
def on_button_click(b):
    global value  # 必要であればグローバル変数として宣言
    global data
    global input_list
    with output_area:
        output_area.clear_output()
        file_path = array_input.value  # テキストボックスから現在の入力を取得
        #value = file_path
        data = pd.read_csv(file_path)  
        # アミノ酸配列のみを表から取得
        acid_list = data[['CycPeptMPDB_ID','Sequence']]

        print(f"Sequence の最初の要素: {acid_list.iloc[0]}")
        print(f"Sequence の最初の要素の型: {type(acid_list.iloc[0])}")
        # 必要な変数をここで定義
        segid = "PROA"
        rename_str = "rename_str_example"
        delete_str = "delete_str_example"
        cycpep = "cycpep_example"
        kind = ['G','A','V','L','I','C','M','S','T','D','E','N','Q','R',
                'K','H','F','Y','W','P','dA','dV','dL','dI','dC','dM',
                'dS','dT','dD','dE','dN','dQ','dR','dK','dH','dF','dY','dW','dP']
        #アミノ酸配列がリストに含まれていなかったら除外するコード
        for row in acid_list.itertuples(index=False):
            seq_list = ast.literal_eval(row.Sequence)
            seq_id = row.CycPeptMPDB_ID
            if all(acid in kind for acid in seq_list):
                print(f"Processing ID {seq_id} with sequence: {seq_list}")
                input_list = seq_list
                value = seq_id
                try:

In [1]:
#inの結果をin.csvに入れるコマンド
import pandas as pd
import os

def update_csv(value):
    # 設定
    csv_list = ['coul_ener', 'hbond', 'van_der_ener', 'solv_free_ener', 'surf_area', 'polar_surf_area']
    csv_experiment_data = ['クーロン相互作用エネルギー', '分子内水素結合数', 'ファンデルワールスエネルギー', '溶媒和自由エネルギー', '全体の溶媒アクセス表面積', '極性溶媒アクセス表面積']
    csv_file = "/Users/teraimao/experiment/data/csv_data/experiment_data_sur.csv"

    # 値を格納する辞書
    results = {}

    # 必要なディレクトリとファイルを確認・作成
    for j, column_name in zip(csv_list, csv_experiment_data):
        result_file = f"/Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_sur/{j}/average/result.txt"
        dir_path = os.path.dirname(result_file)
        os.makedirs(dir_path, exist_ok=True)  # 必要なディレクトリを作成

        if not os.path.exists(result_file):  # ファイルが存在しない場合は作成
            with open(result_file, 'w') as file:
                file.write("0.0\n")  # 初期値として 0.0 を記録

        # ファイルを読み取り、結果を辞書に保存
        try:
            with open(result_file, 'r') as file:
                results[column_name] = file.readline().strip()  # 最初の行を取得
        except Exception as e:
            print(f"Error reading result file {result_file}: {e}")
            results[column_name] = None

    # CSV ファイルの読み取り
    try:
        df = pd.read_csv(csv_file)
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return

    # `CycPeptMPDB_ID` の存在確認
    if 'CycPeptMPDB_ID' not in df.columns:
        print("Error: 'CycPeptMPDB_ID' column is missing in the CSV file.")
        return

    if value not in df['CycPeptMPDB_ID'].values:
        print(f"Error: {value} not found in 'CycPeptMPDB_ID' column.")
        return

    # 必要な列を追加
    for column_name in csv_experiment_data:
        if column_name not in df.columns:
            df[column_name] = None

    # 結果を設定
    for column_name, result_value in results.items():
        df.loc[df['CycPeptMPDB_ID'] == value, column_name] = result_value

    # CSV ファイルの保存
    try:
        df.to_csv(csv_file, index=False)
        print(f"Updated CSV file saved: {csv_file}")
    except Exception as e:
        print(f"Error saving updated CSV file: {e}")


In [1]:
import subprocess
import os
import paramiko
import ipywidgets as widgets
from IPython.display import display

# テキストボックスの作成
array_input = widgets.Text(
    value='',
    placeholder='配列を入力してください',
    description='配列:',
)

# ボタンの作成
run_button = widgets.Button(
    description='実行',
    button_style='success'
)

# 出力エリアの作成
output_area = widgets.Output()

# テキストボックスとボタン、出力エリアの表示
display(array_input, run_button, output_area)

# 実行ボタンが押されたときに呼び出される関数
def on_button_click(b):
    global input_list  # 必要であればグローバル変数として宣言
    global value
    with output_area:
        output_area.clear_output()
        input_text = array_input.value  # テキストボックスから現在の入力を取得
        value = input_text
        input_list = input_text.split(',')  # カンマで分割してリスト化
        print(f'入力された値: {value}')
        print(f'リスト化：{input_list}')
        
        # 必要な変数をここで定義
        segid = "PROA"
        rename_str = "rename_str_example"
        delete_str = "delete_str_example"
        cycpep = "cycpep_example"

        try:
            copy_seed_work(value)
        except NameError:
            print('Error: copy_seed_work 関数が定義されていません。')
        except Exception as e:
            print(f'An error occurred in copy_seed_work: {e}')

        try:
            copy_seed_pdb(value, input_list)
        except NameError:
            print('Error: copy_seed_pdb 関数が定義されていません。')
        except Exception as e:
            print(f'An error occurred in copy_seed_pdb: {e}')

        try:
            rename_pdb_atom(input_list)
        except NameError:
            print('Error: rename_pdb_atom 関数が定義されていません。')
        except Exception as e:
            print(f'An error occurred in rename_pdb_atom: {e}')

        try:
            make_rename_tcl(input_list, value)
        except NameError:
            print('Error: make_rename_tcl 関数が定義されていません。')
        except Exception as e:
            print(f'An error occurred in make_rename_tcl: {e}')
                
        try:
            seed_charmm(input_list)
        except NameError:
            print("Error: seed_charmm 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in seed_charmm: {e}")

        try:
            in_charminp(value)
        except NameError:
            print("Error: in_charminp 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in in_charminp: {e}")

        try:
            gbsw_in_charmm()
        except NameError:
            print("Error: gbsw_in_charmm 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in gbsw_in_charmm: {e}")

        try:
            gbsw_sur_charmm()
        except NameError:
            print("Error: gbsw_sur_charmm 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in gbsw_sur_charmm: {e}")

        try:
            gbsw_out_charmm()
        except NameError:
            print("Error: gbsw_out_charmm 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in gbsw_out_charmm: {e}")

        try:
            in_gbsw_in(value)
        except NameError:
            print("Error: in_gbsw_in 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in in_gbsw_in: {e}")

        try:
            in_gbsw_sur(value)
        except NameError:
            print("Error: in_gbsw_sur 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in in_gbsw_sur: {e}")

        try:
            in_gbsw_out(value)
        except NameError:
            print("Error: in_gbsw_out 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in in_gbsw_out: {e}")
        
        try:
            connect_server()
        except NameError:
            print("Error: connect_server 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in connect_server: {e}")

        try:
            send_dir(value)
        except NameError:
            print("Error: send_dir 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in send_dir: {e}")
        
        try:
            run_sh_gbsw(value)
        except NameError:
            print("Error: send_dir 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in send_dir: {e}")

# ボタンがクリックされたときに実行されるように設定
run_button.on_click(on_button_click)


Text(value='', description='配列:', placeholder='配列を入力してください')

Button(button_style='success', description='実行', style=ButtonStyle())

Output()

In [2]:
import pandas as pd
import ast
# テキストボックスの作成
array_input = widgets.Text(
    value='',
    placeholder='配列を入力してください',
    description='配列:',
)

# ボタンの作成
run_button = widgets.Button(
    description='実行',
    button_style='success'
)

# 出力エリアの作成
output_area = widgets.Output()

# テキストボックスとボタン、出力エリアの表示
display(array_input, run_button, output_area)

# 実行ボタンが押されたときに呼び出される関数
def on_button_click(b):
    global value  # 必要であればグローバル変数として宣言
    global data
    global input_list
    with output_area:
        output_area.clear_output()
        file_path = array_input.value  # テキストボックスから現在の入力を取得
        #value = file_path
        data = pd.read_csv(file_path)  
        # アミノ酸配列のみを表から取得
        acid_list = data[['CycPeptMPDB_ID','Sequence']]

        print(f"Sequence の最初の要素: {acid_list.iloc[0]}")
        print(f"Sequence の最初の要素の型: {type(acid_list.iloc[0])}")
        # 必要な変数をここで定義
        segid = "PROA"
        rename_str = "rename_str_example"
        delete_str = "delete_str_example"
        cycpep = "cycpep_example"
        kind = ['G','A','V','L','I','C','M','S','T','D','E','N','Q','R',
                'K','H','F','Y','W','P','dA','dV','dL','dI','dC','dM',
                'dS','dT','dD','dE','dN','dQ','dR','dK','dH','dF','dY','dW','dP']
        #アミノ酸配列がリストに含まれていなかったら除外するコード
        for row in acid_list.itertuples(index=False):
            seq_list = ast.literal_eval(row.Sequence)
            seq_id = row.CycPeptMPDB_ID
            if all(acid in kind for acid in seq_list):
                print(f"Processing ID {seq_id} with sequence: {seq_list}")
                input_list = seq_list
                value = seq_id
                try:
                    copy_seed_work(value)
                except NameError:
                    print('Error: copy_seed_work 関数が定義されていません。')
                except Exception as e:
                    print(f'An error occurred in copy_seed_work: {e}')
                try:
                    copy_seed_pdb(value, input_list)
                except NameError:
                    print('Error: copy_seed_pdb 関数が定義されていません。')
                except Exception as e:
                    print(f'An error occurred in copy_seed_pdb: {e}')

                try:
                    rename_pdb_atom(input_list)
                except NameError:
                    print('Error: rename_pdb_atom 関数が定義されていません。')
                except Exception as e:
                    print(f'An error occurred in rename_pdb_atom: {e}')

                try:
                    make_rename_tcl(input_list, value)
                except NameError:
                    print('Error: make_rename_tcl 関数が定義されていません。')
                except Exception as e:
                    print(f'An error occurred in make_rename_tcl: {e}')
                
                try:
                    seed_charmm(input_list)
                except NameError:
                    print("Error: seed_charmm 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in seed_charmm: {e}")

                try:
                    in_charminp(value)
                except NameError:
                    print("Error: in_charminp 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in in_charminp: {e}")

                try:
                    gbsw_in_charmm()
                except NameError:
                    print("Error: gbsw_in_charmm 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in gbsw_in_charmm: {e}")
                
                try:
                    gbsw_sur_charmm()
                except NameError:
                    print("Error: gbsw_sur_charmm 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in gbsw_sur_charmm: {e}")

                try:
                    gbsw_out_charmm()
                except NameError:
                    print("Error: gbsw_out_charmm 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in gbsw_out_charmm: {e}")

                try:
                    in_gbsw_in(value)
                except NameError:
                    print("Error: in_gbsw_in 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in in_gbsw_in: {e}")

                try:
                    in_gbsw_sur(value)
                except NameError:
                    print("Error: in_gbsw_sur 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in in_gbsw_sur: {e}")

                try:
                    in_gbsw_out(value)
                except NameError:
                    print("Error: in_gbsw_out 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in in_gbsw_out: {e}")
        
                try:
                    connect_server()
                except NameError:
                    print("Error: connect_server 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in connect_server: {e}")

                try:
                    send_dir(value)
                except NameError:
                    print("Error: send_dir 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in send_dir: {e}")
        
                try:
                    run_sh_gbsw(value)
                except NameError:
                    print("Error: send_dir 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in send_dir: {e}")

        

# ボタンがクリックされたときに実行されるように設定
run_button.on_click(on_button_click)

Text(value='', description='配列:', placeholder='配列を入力してください')

Button(button_style='success', description='実行', style=ButtonStyle())

Output()

In [1]:
import pandas as pd
import ast
# テキストボックスの作成
array_input = widgets.Text(
    value='',
    placeholder='表のディレクトリパスを入力してください',
    description='パス:',
)

# ボタンの作成
run_button = widgets.Button(
    description='実行',
    button_style='success'
)

# 出力エリアの作成
output_area = widgets.Output()

# テキストボックスとボタン、出力エリアの表示
display(array_input, run_button, output_area)

# 実行ボタンが押されたときに呼び出される関数
def on_button_click(b):
    global value  # 必要であればグローバル変数として宣言
    global data
    global input_list
    with output_area:
        output_area.clear_output()
        file_path = array_input.value  # テキストボックスから現在の入力を取得
        #value = file_path
        data = pd.read_csv(file_path)  
        # アミノ酸配列のみを表から取得
        acid_list = data[['CycPeptMPDB_ID','Sequence']]

        print(f"Sequence の最初の要素: {acid_list.iloc[0]}")
        print(f"Sequence の最初の要素の型: {type(acid_list.iloc[0])}")
        # 必要な変数をここで定義
        segid = "PROA"
        rename_str = "rename_str_example"
        delete_str = "delete_str_example"
        cycpep = "cycpep_example"
        kind = ['G','A','V','L','I','C','M','S','T','D','E','N','Q','R',
                'K','H','F','Y','W','P','dA','dV','dL','dI','dC','dM',
                'dS','dT','dD','dE','dN','dQ','dR','dK','dH','dF','dY','dW','dP',
               'Abu','Sar','Me_Bmt(E)','meL','meV','meA','meI','meM','meS','meT',
                'meD','meN','meQ','meK','meH','meF','meY','meW']
        #アミノ酸配列がリストに含まれていなかったら除外するコード
        for row in acid_list.itertuples(index=False):
            seq_list = ast.literal_eval(row.Sequence)
            seq_id = row.CycPeptMPDB_ID
            if all(acid in kind for acid in seq_list):
                print(f"Processing ID {seq_id} with sequence: {seq_list}")
                input_list = seq_list
                value = seq_id
                
                try:
                    copy_seed_work(value)
                except NameError:
                    print('Error: copy_seed_work 関数が定義されていません。')
                except Exception as e:
                    print(f'An error occurred in copy_seed_work: {e}')
                try:
                    copy_seed_pdb(value, input_list)
                except NameError:
                    print('Error: copy_seed_pdb 関数が定義されていません。')
                except Exception as e:
                    print(f'An error occurred in copy_seed_pdb: {e}')

                try:
                    rename_pdb_atom(input_list)
                except NameError:
                    print('Error: rename_pdb_atom 関数が定義されていません。')
                except Exception as e:
                    print(f'An error occurred in rename_pdb_atom: {e}')

                try:
                    make_rename_tcl(input_list, value)
                except NameError:
                    print('Error: make_rename_tcl 関数が定義されていません。')
                except Exception as e:
                    print(f'An error occurred in make_rename_tcl: {e}')
                
                try:
                    seed_charmm(input_list)
                except NameError:
                    print("Error: seed_charmm 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in seed_charmm: {e}")

                try:
                    in_charminp(value)
                except NameError:
                    print("Error: in_charminp 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in in_charminp: {e}")

                try:
                    gbsw_in_charmm(input_list)
                except NameError:
                    print("Error: gbsw_in_charmm 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in gbsw_in_charmm: {e}")
                
                try:
                    gbsw_sur_charmm(input_list)
                except NameError:
                    print("Error: gbsw_sur_charmm 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in gbsw_sur_charmm: {e}")

                try:
                    gbsw_out_charmm(input_list)
                except NameError:
                    print("Error: gbsw_out_charmm 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in gbsw_out_charmm: {e}")

                try:
                    in_gbsw_in(value)
                except NameError:
                    print("Error: in_gbsw_in 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in in_gbsw_in: {e}")

                try:
                    in_gbsw_sur(value)
                except NameError:
                    print("Error: in_gbsw_sur 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in in_gbsw_sur: {e}")

                try:
                    in_gbsw_out(value)
                except NameError:
                    print("Error: in_gbsw_out 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in in_gbsw_out: {e}")
        
                try:
                    connect_server()
                except NameError:
                    print("Error: connect_server 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in connect_server: {e}")

                try:
                    send_dir(value)
                except NameError:
                    print("Error: send_dir 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in send_dir: {e}")
        
                try:
                    run_sh_gbsw(value)
                except NameError:
                    print("Error: run_sh_gbsw 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in run_sh_gbsw: {e}")

                try:
                    run_sh_hbond(value)
                except NameError:
                    print("Error: run_sh_gbsw 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in run_sh_hbond: {e}")
                    
                try:
                    return_copy_work(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in return_copy_work: {e}")
                    
                try:
                    summarize_solv_free_ene(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in summarize_solv_free_ene: {e}")
                    
                try:
                    summarize_van_der_ener(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in summarize_van_der_ener: {e}")
                    
                try:
                    summarize_coul_ener(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in summarize_coul_ener: {e}")
    
                try:
                    calculate_column_average(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in calculate_column_average: {e}")
                
                try:
                    calculate_hbond_average(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in calculate_hbond_average: {e}")
    
                try:
                    save_results(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in save_results: {e}")
    
                try:
                    in_csv(file_path)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in in_csv: {e}")
                
                try:
                    update_csv_in(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in update_csv_in: {e}")
                
                try:
                    update_csv_sur(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in update_csv_sur: {e}")
                
                try:
                    update_csv_out(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in update_csv_out: {e}")


        

                
# ボタンがクリックされたときに実行されるように設定
run_button.on_click(on_button_click)

NameError: name 'widgets' is not defined